In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from pandas.plotting import register_matplotlib_converters
import japanize_matplotlib
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

# yfinanceを使用して日経225のデータを取得
NK225_data = yf.download('^N225', start='任意の日付', end=datetime.now().strftime('%Y-%m-%d'))

# 'Adj Close'列のデータを取得
NK225_prices = NK225_data['Adj Close'].dropna()

# データをCSVファイルに保存
NK225_prices.to_csv('NK225_prices.csv')

# CSVファイルからデータを読み込む
NK225_prices = pd.read_csv('NK225_prices.csv', index_col='Date', parse_dates=True)

# データの正規化
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(np.array(NK225_prices).reshape(-1, 1))

# データの整形（LSTMに入力する形に変換）
def create_dataset(dataset, time_steps=1):
    X, y = [], []
    for i in range(len(dataset)-time_steps):
        a = dataset[i:(i+time_steps), 0]
        X.append(a)
        y.append(dataset[i + time_steps, 0])
    return np.array(X), np.array(y)

time_steps = 任意の数値  # 適切なタイムステップを選択する必要
X, y = create_dataset(data_scaled, time_steps)

# データの分割（トレーニングデータとテストデータ）
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# ニューラルネットワークの構築
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(time_steps, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# ニューラルネットワークのトレーニング
model.fit(X_train, y_train, epochs=200, batch_size=32, verbose=2)

# テストデータに対する予測
test_input = data_scaled[-time_steps:].reshape((1, time_steps, 1))
predicted_value = model.predict(test_input)
predicted_value = scaler.inverse_transform(predicted_value)

print("未来の予測値:")
print(predicted_value)

# 実際のデータと予測データの可視化
plt.figure(figsize=(10, 6))
plt.plot(NK225_prices.index, NK225_prices['Adj Close'], label='実際のデータ')
plt.plot(NK225_prices.index[-1] + timedelta(days=1), predicted_value[0, 0], marker='o', color='red', label='未来の予測値')
plt.title('日経225株価予測')
plt.xlabel('日付')
plt.ylabel('調整後終値')
plt.legend()
plt.show()